In [ ]:
from nimlab import datasets as nimds
import numpy as np
from nilearn import image, plotting, maskers
import nibabel as nib
import os
import pandas as pd
import glob

## Identify Niftis to Use as Maps
### Organize Maps for Utilization in a Folder

In [ ]:
#CHANGE EVERY RUN
file_pattern  = '*07*.nii'


In [ ]:
#User Input Below
map_dir = r"C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\functional_networks\niftis_to_overlap"
analysis = "rios_r_map"
# map_path2 = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\functional_networks\niftis_to_overlap\memory_network_T_map.nii'
out_dir = r"C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\functional_networks\thresholded_networks"

In [ ]:
#preparing
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

glob_path = os.path.join(map_dir, file_pattern)
print('Path: \n', glob_path)
globbed = glob.glob(glob_path)
for i in range(0, len(globbed)):
    print(f'\n Nifti {i} found: \n', globbed[i])

mask = nimds.get_img("mni_icbm152")

In [ ]:
def z_score_matrix(matrix):
    from scipy.stats import zscore
    print('--------------------------------Performing z_score--------------------------------')
    print('pre z score max: ', np.max(matrix), np.shape(matrix))
    z_matrix = zscore(matrix)
    z_matrix = np.nan_to_num(np.nan_to_num(z_matrix, nan=0, posinf=0, neginf=0))
    print('post z score max: ', np.max(z_matrix), np.shape(z_matrix))
    return z_matrix

In [ ]:
def threshold_matrix(matrix, threshold=0.5, probability=True, direction='keep_greater'):
    import scipy.stats as st
    print('--------------------------------Performing threshold--------------------------------')
    if probability == True:
        threshold = st.norm.ppf(threshold) #this generates the z score that indicates everything BELOW the threshold value
        print('z score threshold is: ', threshold)
    if direction == 'keep_greater':
        print('pre threshold: ', np.count_nonzero(matrix))
        print('max z score in matrix is: ', np.max(matrix))
        print('min z score in matrix is: ', np.min(matrix))
        empty = np.ones(matrix.shape)
        empty[matrix < threshold] = 0
        matrix = matrix * empty
        print('post threshold: ', np.count_nonzero(matrix))
        print('max z score in matrix is: ', np.max(matrix))
        print('min z score in matrix is: ', np.min(matrix))
        print('I zero everything below threshold')
    elif direction == 'keep_lesser':
        matrix[matrix > st.norm.ppf(threshold)] = 0
        print('I will zero everything above threshold')
    return matrix

In [ ]:
def overlap(mask_matrix=globbed, base_matrix=r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\functional_networks\niftis_to_overlap\Fx-DBS-Network_N46.nii', 
            mask=False, dozscore=False, do_threshold=False, threshold=0.95, give_og=False):
    #import the matrix and prepare it.
    img = image.load_img(base_matrix); mx = img.get_fdata(); mx = np.nan_to_num(mx, nan=0, posinf=10, neginf=-10)
    if dozscore:
        mx = z_score_matrix(mx)
    if do_threshold:
        mx = threshold_matrix(mx, threshold=threshold)
        og_mx=mx
    if dozscore & do_threshold == False: #Just normalize the matrix instead.
        overlap_mx = []
        if overlap_mx == []:
            overlap_mx = np.zeros(mx.shape) #generate empty matric
            overlap_mx = overlap_mx + mx/np.max(abs(mx)) #Fill empty matrix with normalized values from the path matrix
    else:
        overlap_mx = mx

    #Perform overlapping process.     
    if mask == True:
        for path in globbed:
            img = image.load_img(path); mx = img.get_fdata(); mx = np.nan_to_num(mx, nan=0, posinf=0, neginf=0) #Import the target mask image from path in glob
            overlap_mx[mx==0] = 0
            print('Max value in mask: ', np.max(overlap_mx))

    if dozscore & do_threshold == False:
        return overlap_mx/np.max(abs(overlap_mx))
    elif give_og==True:
        print('OG Returned')
        return og_mx
    else:
        return overlap_mx

## Create Maps

In [ ]:
threshold = 0.999
##----------------------------------------------------------------User Input Above----------------------------------------------------------------
og_mx  = overlap(globbed, dozscore=True, do_threshold=True, threshold=threshold, mask=False, give_og=True)

ovr_img1 = image.new_img_like(mask, og_mx)
ovr_html1 = plotting.view_img(ovr_img1, cut_coords=(0,0,0), title=(f'unmasked&thresh zscore: {threshold}'), black_bg=False, opacity=.75, cmap='ocean_hot')


In [ ]:
overlap_mx = overlap(globbed, dozscore=True, do_threshold=True, threshold=threshold, mask=True)

ovr_img2 = image.new_img_like(mask, overlap_mx)
ovr_html2 = plotting.view_img(ovr_img2, cut_coords=(0,0,0), title=(f'masked&thresh prob: {threshold}'), black_bg=False, opacity=.75, cmap='ocean_hot')


In [ ]:
ovr_html1

In [ ]:
ovr_html2

In [ ]:
#User Input Below
try:   
    maskname = os.path.basename(globbed[0]).split('.nii')[0]
    analysis2 = analysis.split('.nii')[0] + '_masked_by_' + maskname
    print('Title: ' + analysis2)
except:
    maskname = os.path.basename(globbed[0]).split('.nii')[0]
    analysis2 = analysis + '_masked_by_' + maskname
    print('Title: ' + analysis2)
    print('error')

In [ ]:
#Save
threshold = str(threshold).split(".")[1]
print(threshold)
ovr_img1.to_filename(os.path.join(out_dir, f'{analysis}_ztrans_and_threshold_prob_{threshold}'))
ovr_html1.save_as_html(os.path.join(out_dir, f'{analysis}_ztrans_and_threshold_prob_{threshold}.html'))
ovr_img2.to_filename(os.path.join(out_dir, f'{analysis2}_masked&threshprob_{threshold}'))
ovr_html2.save_as_html(os.path.join(out_dir, f'{analysis2}_masked&threshprob_{threshold}.html'))

print('saved to: ', out_dir)